# Data Preprocessing

## Converting from txt to csv and adding column names

In [27]:
import pandas as pd

In [28]:
# Define the column specifications
col_specs = [(0, 4), (4, 6), (6, 8), (8, 11), (11, 13), (13, 14), (14, 15), (15, 16), (16, 18), (18, 26)]
col_names = [
    'Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code', 
    'Registry', 'Race', 'Origin', 'Sex', 'Age', 'Population'
]

In [29]:
# Read the fixed-width file
df = pd.read_fwf('demographic_data.txt', colspecs=col_specs, header=None, names=col_names)

## Separating datasets by Age and Sex

In [31]:
data = df.copy()
data.head(), data.columns

(   Year State Abbreviation  State FIPS Code  County FIPS Code  Registry  Race  \
 0  1990                 AL                1                 1        99     1   
 1  1990                 AL                1                 1        99     1   
 2  1990                 AL                1                 1        99     1   
 3  1990                 AL                1                 1        99     1   
 4  1990                 AL                1                 1        99     1   
 
    Origin  Sex  Age  Population  
 0       0    1    0         239  
 1       0    1    1         821  
 2       0    1    2        1089  
 3       0    1    3        1144  
 4       0    1    4        1046  ,
 Index(['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code',
        'Registry', 'Race', 'Origin', 'Sex', 'Age', 'Population'],
       dtype='object'))

In [32]:
dataset_by_sex = data[['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code', 'Sex', 'Population']]
dataset_by_age = data[['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code', 'Age', 'Population']]

## Aggregating rows

In [33]:
# Group by specified columns and sum the population
aggregated_data = dataset_by_sex.groupby(['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code', 'Sex']).agg(
    Total_Population=pd.NamedAgg(column='Population', aggfunc='sum')
).reset_index()

# Display the aggregated dataset
aggregated_data.head()

,Year,State Abbreviation,State FIPS Code,County FIPS Code,Sex,Total_Population
0,1990,AK,2,10,1,7943
1,1990,AK,2,10,2,4133
2,1990,AK,2,20,1,117036
3,1990,AK,2,20,2,110578
4,1990,AK,2,50,1,7170


In [34]:
# Save the aggregated dataset as a CSV file
aggregated_csv_path = 'aggregated_dataset_by_sex.csv'
aggregated_data.to_csv(aggregated_csv_path, index=False)

# Provide the path to the saved file
aggregated_csv_path

'aggregated_dataset_by_sex.csv'

In [35]:
# Group by specified columns and sum the population
aggregated_data_by_age = dataset_by_age.groupby(['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code', 'Age']).agg(
    Total_Population=pd.NamedAgg(column='Population', aggfunc='sum')
).reset_index()

# Display the aggregated dataset
aggregated_data_by_age.head()

,Year,State Abbreviation,State FIPS Code,County FIPS Code,Age,Total_Population
0,1990,AK,2,10,0,211
1,1990,AK,2,10,1,739
2,1990,AK,2,10,2,727
3,1990,AK,2,10,3,515
4,1990,AK,2,10,4,749


In [36]:
# Save the aggregated dataset by Age as a CSV file
aggregated_by_age_csv_path = 'aggregated_dataset_by_age.csv'
aggregated_data_by_age.to_csv(aggregated_by_age_csv_path, index=False)

# Provide the path to the saved file
aggregated_by_age_csv_path

'aggregated_dataset_by_age.csv'

## Creating New Features - Percentage and Reducing Redunduncy

## 1. Sex

In [37]:
import pandas as pd
df = pd.read_csv('aggregated_dataset_by_sex.csv')
print(df.dtypes)

Year                   int64
State Abbreviation    object
State FIPS Code        int64
County FIPS Code       int64
Sex                    int64
Total_Population       int64
dtype: object


In [38]:
df['Sex'] = df['Sex'].map({1: 'Male', 2: 'Female'})
df_pivot = df.pivot_table(index=['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code'], 
                          columns='Sex', 
                          values='Total_Population', 
                          aggfunc='sum').reset_index()

df_pivot['Total_Population'] = df_pivot['Female'] + df_pivot['Male']

In [39]:
df_pivot['percentage_male'] = (df_pivot['Male'] / df_pivot['Total_Population'] * 100).round(2)
df_pivot['percentage_female'] = (df_pivot['Female'] / df_pivot['Total_Population'] * 100).round(2)

df_pivot.columns = ['year', 'state_abbr', 'state_fips', 'county_fips', 'female_pop', 'male_pop', 'total_population', 'percentage_male', 'percentage_female']
df_pivot.to_csv('demographic_data_sex.csv', index=False)

print(df_pivot.head())

   year state_abbr  state_fips  county_fips  female_pop  male_pop  \
0  1990         AK           2           10        4133      7943   
1  1990         AK           2           20      110578    117036   
2  1990         AK           2           50        6547      7170   
3  1990         AK           2           60         559       829   
4  1990         AK           2           68         712      1013   

   total_population  percentage_male  percentage_female  
0             12076            65.78              34.22  
1            227614            51.42              48.58  
2             13717            52.27              47.73  
3              1388            59.73              40.27  
4              1725            58.72              41.28  


## 2. Age

In [40]:
import pandas as pd

# Load your data
df = pd.read_csv('aggregated_dataset_by_age.csv')

# Define age category ranges
age_categories = {
    "under_18_pop": range(0, 4),
    "18_44_pop": range(4, 10),
    "45_64_pop": range(10, 14),
    "above_65_pop": range(14, 19)
}

In [41]:
# Initialize columns for population sums
for category in age_categories:
    df[category] = 0

# Sum populations within each age range for each row in the dataframe
for index, row in df.iterrows():
    for category, age_range in age_categories.items():
        if row['Age'] in age_range:
            df.loc[index, category] += row['Total_Population']

# Drop the 'Age' and 'Total_Population' columns as they are no longer needed
df.drop(['Age', 'Total_Population'], axis=1, inplace=True)

# Remove duplicates after aggregation, to prevent repeating sums
df = df.drop_duplicates()

In [42]:
grouped = df.groupby(['Year', 'State Abbreviation', 'State FIPS Code', 'County FIPS Code']).sum().reset_index()

grouped.rename(columns={
    "Year": "year",
    "State Abbreviation": "state_abbr",
    "State FIPS Code": "state_fips",
    "County FIPS Code": "county_fips"
}, inplace=True)

grouped['total_population'] = grouped[['under_18_pop', '18_44_pop', '45_64_pop', 'above_65_pop']].sum(axis=1)
for category in age_categories.keys():
    grouped[f'percentage_{category}'] = (grouped[category] / grouped['total_population'] * 100).round(2)

In [43]:
grouped.to_csv('demographic_data_age.csv', index=False)

print(grouped.head())

   year state_abbr  state_fips  county_fips  under_18_pop  18_44_pop  \
0  1990         AK           2           10          2192       8556   
1  1990         AK           2           20         59704     124598   
2  1990         AK           2           50          4732       6524   
3  1990         AK           2           60           337        819   
4  1990         AK           2           68           418        908   

   45_64_pop  above_65_pop  total_population  percentage_under_18_pop  \
0       1201           127             12076                    18.15   
1      34972          8340            227614                    26.23   
2       1824           637             13717                    34.50   
3        199            33              1388                    24.28   
4        339            60              1725                    24.23   

   percentage_18_44_pop  percentage_45_64_pop  percentage_above_65_pop  
0                 70.85                  9.95          